### 自动化抓取健康信息短视频

- 技术栈
    proxy_pool, 
    redis,
    python...
- 预期效果：
    1)输入关键词，自动搜索相关视频并进行下载
    2)提取检索视频的作者，再下载作者主页下的全部作品

In [1]:
'''
import redis as redis
import numpy as np
#实现一个连接池
pool = redis.ConnectionPool(host='127.0.0.1',port=6379,password='123456')   
r = redis.Redis(connection_pool=pool)
proxy_dict = r.hgetall(name='use_proxy')
proxy_keys= list(proxy_dict.keys())
this_key =random.choice(proxy_keys)
print(this_key)
'''

"\nimport redis as redis\nimport numpy as np\n#实现一个连接池\npool = redis.ConnectionPool(host='127.0.0.1',port=6379,password='123456')   \nr = redis.Redis(connection_pool=pool)\nproxy_dict = r.hgetall(name='use_proxy')\nproxy_keys= list(proxy_dict.keys())\nthis_key =random.choice(proxy_keys)\nprint(this_key)\n"

#### 定义函数

In [2]:
import redis as redis
import requests
import random
import numpy as np
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
pool = redis.ConnectionPool(host='127.0.0.1',port=6379,password='123456')   

global false, null, true,headers
false = False
null = ''
true = True
headers = {

    'naive_header':{
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'cookie':'douyin.com; ttwid=1%7CkUxzTmMLz56Qi8ZFSM5oAeco3xt7h1G3Zm2pTOvDJ94%7C1659753544%7C9e46280257ebea6ea7a75bd54e1d3a1eb8275a6f7e9573873adff473d1b43729; s_v_web_id=verify_l6haf2io_DEf1VsiL_5ZAt_4ZT3_9mmq_BPKClfkvzteG; passport_csrf_token=8f7f807cebb138e41db5c45de48e40a5; passport_csrf_token_default=8f7f807cebb138e41db5c45de48e40a5; n_mh=l3a4V-yOuRsRcww8xXNwk34cchF8GuWclHTor510pkM; sso_uid_tt=85637d575326a0487c45a7457cd05278; sso_uid_tt_ss=85637d575326a0487c45a7457cd05278; toutiao_sso_user=69cbe3824457428c9d17f931af627e5e; toutiao_sso_user_ss=69cbe3824457428c9d17f931af627e5e; sid_ucp_sso_v1=1.0.0-KDQzNTNmNDYzMmJhODMwODE0NzFhM2JiMzc0YmJjZDRiMmY0ZDJiNjUKHwjIl9CZ3Y2VAhCO6NKXBhjvMSAMMLzn9JYGOAZA9AcaAmxmIiA2OWNiZTM4MjQ0NTc0MjhjOWQxN2Y5MzFhZjYyN2U1ZQ; ssid_ucp_sso_v1=1.0.0-KDQzNTNmNDYzMmJhODMwODE0NzFhM2JiMzc0YmJjZDRiMmY0ZDJiNjUKHwjIl9CZ3Y2VAhCO6NKXBhjvMSAMMLzn9JYGOAZA9AcaAmxmIiA2OWNiZTM4MjQ0NTc0MjhjOWQxN2Y5MzFhZjYyN2U1ZQ; odin_tt=ee963c14296dab5d79e7d98749f77ba541b3b2c4487dbf52977d93d0ebfb52ccefd6516964bb0473ac4a1057ac9a4d947d13fa5b76c98e0dd78b08cdb55d1483; sid_guard=69cbe3824457428c9d17f931af627e5e%7C1660204046%7C5184000%7CMon%2C+10-Oct-2022+07%3A47%3A26+GMT; uid_tt=85637d575326a0487c45a7457cd05278; uid_tt_ss=85637d575326a0487c45a7457cd05278; sid_tt=69cbe3824457428c9d17f931af627e5e; sessionid=69cbe3824457428c9d17f931af627e5e; sessionid_ss=69cbe3824457428c9d17f931af627e5e; sid_ucp_v1=1.0.0-KDQzNTNmNDYzMmJhODMwODE0NzFhM2JiMzc0YmJjZDRiMmY0ZDJiNjUKHwjIl9CZ3Y2VAhCO6NKXBhjvMSAMMLzn9JYGOAZA9AcaAmxmIiA2OWNiZTM4MjQ0NTc0MjhjOWQxN2Y5MzFhZjYyN2U1ZQ; ssid_ucp_v1=1.0.0-KDQzNTNmNDYzMmJhODMwODE0NzFhM2JiMzc0YmJjZDRiMmY0ZDJiNjUKHwjIl9CZ3Y2VAhCO6NKXBhjvMSAMMLzn9JYGOAZA9AcaAmxmIiA2OWNiZTM4MjQ0NTc0MjhjOWQxN2Y5MzFhZjYyN2U1ZQ; THEME_STAY_TIME=%22298986%22; IS_HIDE_THEME_CHANGE=%221%22; download_guide=%223%2F20220905%22; SEARCH_RESULT_LIST_TYPE=%22single%22; strategyABtestKey=1662611691.699; _tea_utm_cache_2018=undefined; douyin.com; __ac_nonce=063198f590062f5912f1b; __ac_signature=_02B4Z6wo00f011szYvQAAIDAD14XPg-Xr3NbE2ZAALXgUwbZS.d6Y82unOlICU..UGb7Ka5RLNudU9GDLvyE-zUCSkaX9tRqTravEdZ340lMvDX2dNmvRjX1wS-JjhLqD6aejnEIHv5APRl4b1; msToken=xInRPESqOLWXb0h-ktd_U_M5r_48sS89k3DzO4tmrUUelSK7IQWRg7Q6a0Mz_MeKOY_Bzid_GFAGOVQkR8ZlMxwBI8vWNMnUw2D3cemRXDts7XZxjm7mZcMh1i6uPbc=; home_can_add_dy_2_desktop=%221%22; msToken=t9W-C0asGUaLdOK5m2jKF3iT-j1qpzowjiO4-VwkWr0T9cuLyslVyi_Pv4hsEPu4-prI_rWs3QcXsEDkebOSVgSbqTy8RryzWdk2e8z4bzBX7F9dYcHEP3lUEL00cpQ=; tt_scid=4mxPX-10y6mfT.Ej4KXKOP0uQYj6fOSdUmiLRayNEpdbpgV-c32GO5Qv8XBbEl230cfa'
    }
}

def get_proxy():
    r = redis.Redis(connection_pool=pool)
    proxy_keys= list( r.hkeys(name='use_proxy'))
    this_key =random.choice(proxy_keys)
    js = eval(r.hget(name='use_proxy',key=this_key).decode('utf8'))
    return js

def get_https_proxy():
    r = redis.Redis(connection_pool=pool)
    proxy_keys= list(r.hkeys(name='use_proxy'))
    https_proxy = []
    while len(https_proxy)==0 :
        for key in proxy_keys:
            js = eval(r.hget(name='use_proxy',key=key).decode('utf8'))
            if js.get('https') :
                https_proxy.append(key)
    this_key =random.choice(https_proxy)        
    js = eval(r.hget(name='use_proxy',key=this_key).decode('utf8'))
    return js

def delete_proxy(proxy):
    r = redis.Redis(connection_pool=pool)
    r.hdel('use_proxy',proxy)

    
def getHtml(url):
    # ....

    retry_count = 5
    #http代理似乎已经够用
    proxy = get_proxy().get('proxy')
    #proxy = get_https_proxy().get('proxy')
    #print(proxy)
    while retry_count > 0:
        try:
           # html = requests.get('http://demo.spiderpy.cn/',headers = headers.get('naive_header'),timeout=3,proxies={"http": "http://{}".format(proxy)})
            html = requests.get(url=url,headers = headers.get('naive_header'), proxies={"http": "http://{}".format(proxy)},timeout=1, )
            # 使用代理访问
            #print('bar')
            return html
        except Exception:
            retry_count -= 1
            #print('foo')
    # 删除代理池中无效代理
    #delete_proxy(proxy)
    return None

In [3]:
def getVideo(url):
    # ....
    url = 'https://'+url
    retry_count = 5
    #http代理似乎已经够用
    proxy = get_proxy().get('proxy')
    #proxy = get_https_proxy().get('proxy')
    #print(proxy)
    while retry_count > 0:
        try:
           # html = requests.get('http://demo.spiderpy.cn/',headers = headers.get('naive_header'),timeout=3,proxies={"http": "http://{}".format(proxy)})
            video = requests.get(url=url, proxies={"http": "http://{}".format(proxy)},timeout=3, ).content
            # 使用代理访问
            #print('bar')
            return video
        except Exception:
            retry_count -= 1
            #print('foo')
    # 删除代理池中无效代理
    delete_proxy(proxy)
    return None

In [4]:
import requests
import re
import os


def downloadVideo (res):
    #print(res)
    #寻找视频名称
    title = re.findall('<title data-react-helmet="true">(.*)?</title>',res)
    if len(title) >0:
        title = title[0]
    else :
        return
    #寻找下载链接
    #hint:src":"为开头，再到其后第一个"为止，是一个视频链接
    href = re.findall('src%22%3A%22%2F%2F(.*?)%22',res)[1]
    video_url = requests.utils.unquote(href)
    #print(video_url)
    video_content = getVideo(video_url)
    title = re.sub(r'[\\s\\\\/:\\*\\?\\\"<>\\|]','_',title)
    with open ('video'+os.sep+ title +'.mp4', mode = 'wb') as f:
        f.write(video_content)

In [5]:
#非线性地滑动页面
import time
def drop_down(driver):

    curr = 0
    #先加速
    for x in range (9,1000,40):
        pre = curr
        time.sleep(0.1)
        j = x/9
        js = 'document.documentElement.scrollTop = document.documentElement.scrollTop +'+str(j)+';return document.documentElement.scrollTop;'
        curr =driver.execute_script(js)
        if curr == pre:
            break
    #匀速运行指导到达底部        
    retry = 10
    while (pre != curr) and retry >0 :
        pre = curr
        time.sleep(0.1)
        js = 'document.documentElement.scrollTop = document.documentElement.scrollTop +'+str(j)+';return document.documentElement.scrollTop;'
        curr =driver.execute_script(js)
        if pre == curr:
            retry -= 1


In [6]:
def drag_down(driver):    
    driver.implicitly_wait(1)

    for x in range (1,100,40):
        js = 'document.documentElement.scrollTop = document.documentElement.scrollTop +'+str(x)+';return document.documentElement.scrollTop;'
        curr =driver.execute_script(js)
        time.sleep(0.1)

    curr = 0
    #匀速运行指导到达底部        
    while curr == 0:
        time.sleep(0.1)
        js = 'document.documentElement.scrollTop = document.documentElement.scrollTop +'+str(100)+';return document.documentElement.scrollTop;'
        driver.execute_script(js)
        curr = len(driver.find_elements(by= By.LINK_TEXT, value="暂时没有更多了"))


#### 根据www.douyin.com/video/[0-9]+ 找到视频文件并下载到./video中

In [7]:
def down_load_video_from_user_homepage(url):
    url = url
    response = None
    while response == None:
        response = getHtml(url)
    res = response.text
    downloadVideo(res)

In [9]:
url = "https://www.douyin.com/video/7141247266043530500"
res = getHtml(url).text
print(res)

<!DOCTYPE html><html dark="true" ><head><link rel="dns-prefetch" href="//lf1-cdn-tos.bytescm.com"/><link rel="dns-prefetch" href="//lf3-cdn-tos.bytescm.com"/><link rel="dns-prefetch" href="//lf3-static.bytednsdoc.com"/><link rel="dns-prefetch" href="//unpkg.byted-static.com"/><link rel="dns-prefetch" href="//lf1-cdn-tos.bytegoofy.com"/><link rel="dns-prefetch" href="//p3-pc.douyinpic.com"/><link rel="dns-prefetch" href="//p3-pc-sign.douyinpic.com"/><link rel="dns-prefetch" href="//p6-pc-sign.douyinpic.com"/><link rel="dns-prefetch" href="//p9-pc-sign.douyinpic.com"/><link rel="dns-prefetch" href="//verify.snssdk.com"/><link rel="dns-prefetch" href="//mon.snssdk.com"/><link rel="dns-prefetch" href="//sso.douyin.com"/><link rel="dns-prefetch" href="//v26-web.douyinvod.com"/><link rel="dns-prefetch" href="//vcs.snssdk.com"/><script id="RENDER_DATA" type="application/json">%7B%221%22%3A%7B%22envService%22%3A%22prod%22%2C%22abTestData%22%3A%7B%22searchBarPanel%22%3A1%2C%22updateNodeSdk%22%3

#### 网页自动化

In [8]:
import selectors
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By

#url = "https://www.douyin.com/user/MS4wLjABAAAAK1Zd96kIeExggUquf_wrHUPDQYTZfjdGXClxQAJHyMQ"
url = "https://www.douyin.com/user/MS4wLjABAAAA0YYUVgoXmgufnaMmQ7r1Yy-UdzQiIZ-03q3VEZLn-0YrXFSVjrNWy4eIWBVIRjO0"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('blink-settings=imagesEnabled=false')
'''
# 设置代理
proxy = get_https_proxy().get('proxy')

chrome_options.add_argument("-proxy-server=https://{}".format(proxy))
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)')

print(chrome_options.arguments)
'''
preferences = {
    "webrtc.ip_handling_policy": "disable_non_proxied_udp",
    "webrtc.multiple_routes_enabled": False,
    "webrtc.nonproxied_udp_enabled": False
}
chrome_options.add_experimental_option("prefs", preferences)

#打开url
driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe',chrome_options=chrome_options)
#driver.get("chrome://version/",proxies = proxies)
driver.get(url=url)
time.sleep(2)

#关闭登录弹窗
a = driver.find_elements(by= By.CLASS_NAME,value="dy-account-close")
if len(a) >0 :
    a[0].click()
drop_down(driver=driver)
time.sleep(1)
#driver.execute_script('document.documentElement.scrollTop=0') 

#获取页面内容
#hint:先取所有的li标签，再取a标签中的href
#selector = '#root > div > div.T_foQflM > div > div > div > div:nth-child(2) > div.mwo84cvf > div.wwg0vUdQ > div.UFuuTZ1P > ul > li > a'
selector = '#root > div > div > div > div > div > div> div> div> div> ul > li > a'
lis = driver.find_elements(by=By.CSS_SELECTOR,value= selector)
href =[]
for li in lis:
    try:
        href.append(li.get_attribute('href'))
    except Exception:
        continue
driver.quit()
print(len(href))

for url in href:
   down_load_video_from_user_homepage(url)

C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_107556\58040445.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe',chrome_options=chrome_options)
C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_107556\58040445.py:27: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe',chrome_options=chrome_options)


0
